#### SQLite port_lite database: sales, stocks table
#### csv files: daily-watch-list, daily-sales, monitors, stocks-all

In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 7, 16)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-07-16
yesterday: 2023-07-14


### Restart and Run All Cells | daily-watch-list & daily-sales upload to obsidian

In [3]:
#cols = 'fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','chg_acc':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','%chg_acc':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1283104,WHART,2023-07-14,2023-07-14,0,10.70,10.70,0,0.00%,0,0,10.70,10.60,"1,013,328",2023-07-16,2023-07-16,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,11.00,11.00,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,5pct,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-07-14'
ORDER BY a.name



,name
0,AH
1,AWC
2,BBL
3,BCH
4,BDMS
5,BEM
6,BH
7,CPNREIT
8,DIF
9,EA


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-07-14' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('chg_acc = to_price - fm_price',inplace=True)
df['%chg_acc'] = round(df['chg_acc']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,chg_acc,%chg_acc
0,CPNREIT,2023-07-14,2023-07-14,10.90,10.90,"1,206,570",10.90,10.80,B,SET,0.00,0.00%
1,DIF,2023-07-04,2023-07-14,11.00,10.70,"84,070,184",11.20,10.50,B,SET,-0.30,-2.73%
2,GVREIT,2023-07-12,2023-07-14,8.30,8.40,"898,470",8.45,8.20,B,SET,0.10,1.20%
3,JMT,2023-07-13,2023-07-14,34.75,35.50,"47,806,064",35.75,33.25,B,SET50,0.75,2.16%
4,MCS,2023-07-14,2023-07-14,6.35,6.35,"135,358",6.35,6.20,B,SET,0.00,0.00%
5,NER,2023-07-13,2023-07-14,4.46,4.56,"5,529,988",4.58,4.40,B,SET,0.10,2.24%
6,SENA,2023-07-14,2023-07-14,3.00,3.00,"451,000",3.00,2.94,B,SET,0.00,0.00%
7,SINGER,2023-07-13,2023-07-14,8.85,9.10,"11,333,023",9.10,8.70,B,SET100,0.25,2.82%
8,STA,2023-07-13,2023-07-14,17.00,17.00,"9,407,926",17.20,16.60,B,SET100,0.00,0.00%
9,SYNEX,2023-07-14,2023-07-14,11.80,11.80,"171,838",12.00,11.60,B,SET,0.00,0.00%


### Price change up or down more than 3%

In [11]:
# For each row in the DataFrame, generate a series of dates between the start and end dates
df['dates'] = df.apply(lambda x: pd.date_range(x['fm_date'], x['to_date']), axis=1)

# For each row in the DataFrame, select only the business days (Monday through Friday) from the series of dates
df['business_days'] = df['dates'].apply(lambda x: x[x.isin(pd.date_range(x.min(), x.max(), freq='B'))])

# For each row in the DataFrame, count the number of business days between the start and end dates
df['bdays'] = df['business_days'].apply(lambda x: len(x))

In [12]:
colt =  'name %chg_acc status fm_price to_price market \
fm_date bdays max_price min_price'.split()
watch_list = df.loc[abs(df['%chg_acc']) >= 3,colt]
watch_list.sort_values('%chg_acc',ascending=False)
watch_list

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
15,ICHI,3.76,I,13.30,13.80,SET,2023-07-13,2,13.80,12.90
16,JMART,3.14,I,15.90,16.40,SET50,2023-07-13,2,16.70,15.40
23,EA,4.65,O,53.75,56.25,SET50,2023-07-12,3,56.50,52.75
26,PTTEP,4.28,O,152.00,158.50,SET50,2023-07-10,5,159.00,151.00
27,IVL,3.68,S,34.00,35.25,SET50,2023-07-13,2,35.25,33.00
29,PTTGC,3.85,S,39.00,40.50,SET50,2023-07-10,5,40.75,38.50


In [13]:
# cannot use with column that starts with %
#df.query('abs(%chg_acc) >= 1.0')

In [13]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(output_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(data_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(one_file, index=False)
watch_list.sort_values(['%chg_acc'],ascending=[False]).to_csv(osd_file, index=False)

### Create daily-sales from sales

In [14]:
df[colt].style.format(format_dict)

,name,%chg_acc,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
0,CPNREIT,0.00%,B,10.90,10.90,SET,2023-07-14,1,10.90,10.80
1,DIF,-2.73%,B,11.00,10.70,SET,2023-07-04,9,11.20,10.50
2,GVREIT,1.20%,B,8.30,8.40,SET,2023-07-12,3,8.45,8.20
3,JMT,2.16%,B,34.75,35.50,SET50,2023-07-13,2,35.75,33.25
4,MCS,0.00%,B,6.35,6.35,SET,2023-07-14,1,6.35,6.20
5,NER,2.24%,B,4.46,4.56,SET,2023-07-13,2,4.58,4.40
6,SENA,0.00%,B,3.00,3.00,SET,2023-07-14,1,3.00,2.94
7,SINGER,2.82%,B,8.85,9.10,SET100,2023-07-13,2,9.10,8.70
8,STA,0.00%,B,17.00,17.00,SET100,2023-07-13,2,17.20,16.60
9,SYNEX,0.00%,B,11.80,11.80,SET,2023-07-14,1,12.00,11.60


In [15]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(output_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(data_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(one_file, index=False)
df[colt].sort_values(['%chg_acc'],ascending=[True]).to_csv(osd_file, index=False)

### Create monitors from orders

In [16]:
sql = """
SELECT name, trade 
FROM orders 
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.sample(5)

,trade
name,
BH,B
DIF,B
IVL,S
PTTGC,S
PTG,S


In [17]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,18
1,S,12


### output_file = iCloud Drive on PC

In [18]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(output_file)
monitors.to_csv(one_file)

### End of daily process

In [19]:
sql = """
SELECT trade, COUNT(*) AS 'no. of stocks'
FROM orders 
GROUP BY trade
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,trade,no. of stocks
0,B,18
1,S,12


In [20]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
CPNREIT,B,SET
DIF,B,SET
GVREIT,B,SET
JMT,B,SET50
MCS,B,SET
...,...,...
TFG,X,SET
TOP,X,SET50
TTB,X,SET50


In [21]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(one_file)

In [22]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("B")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,trade
name,
BDMS,B
BEM,B
BH,B
CPNREIT,B
DIF,B
EA,B
GVREIT,B
JMT,B
LANNA,B


In [23]:
buy_candidates.shape[0]

18

In [24]:
sql = """
SELECT name, trade 
FROM orders 
WHERE trade IN ("S")
ORDER BY name
"""
sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,trade
name,
AH,S
AWC,S
BBL,S
BCH,S
ICHI,S
IVL,S
JASIF,S
JMART,S
PTG,S


In [25]:
sell_candidates.shape[0]

12